In [114]:
import sqlite3
import pandas as pd
import julian 
import datetime as dt

In [115]:
#Take a look at all the fires.  This may take some time.
con = sqlite3.connect("kaggle_dataset.sqlite")
df = pd.read_sql_query("SELECT * from Fires", con)
con.close();


In [116]:
df.head(20)

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...
5,6,6,FS-1418849,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.635278,-120.103611,5.0,USFS,CA,5,005,Amador,b'\x00\x01\xad\x10\x00\x00\xf0<~\x90\xa1\x06^\...
6,7,7,FS-1418851,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.688333,-120.153333,5.0,USFS,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00$o\x996\xd0\t^\xc0h\...
7,8,8,FS-1418854,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,...,B,40.968056,-122.433889,13.0,STATE OR PRIVATE,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00t)\xe8\xd5\xc4\x9b^\...
8,9,9,FS-1418856,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,...,B,41.233611,-122.283333,13.0,STATE OR PRIVATE,CA,None,None,None,"b'\x00\x01\xad\x10\x00\x00\xdc\x8d\x1e""""\x92^\..."
9,10,10,FS-1418859,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.548333,-120.149167,5.0,USFS,CA,5,005,Amador,b'\x00\x01\xad\x10\x00\x00dS\\\xf2\x8b\t^\xc0\...


In [117]:
sample_data = df.sample(n=20000);

In [118]:
#Write to a shorter .csv for testing purposes
sample_data.to_csv("20k_data.csv")

In [119]:
#Write a .csv that only has our desired target variables
time_cols = ['DISCOVERY_TIME', 'CONT_TIME', 'DISCOVERY_DATE', 'CONT_DATE']
df = df.dropna(subset=time_cols) # remove rows where both of these are missing
df.shape

(892007, 39)

In [120]:
df['DISCOVERY_TIME'].head()

0    1300
1    0845
2    1921
3    1600
4    1600
Name: DISCOVERY_TIME, dtype: object

In [121]:
df['CONT_TIME'].head()

0    1730
1    1530
2    2024
3    1400
4    1200
Name: CONT_TIME, dtype: object

In [122]:
df['DISCOVERY_DATE'].head()

0    2453403.5
1    2453137.5
2    2453156.5
3    2453184.5
4    2453184.5
Name: DISCOVERY_DATE, dtype: float64

In [123]:
df['CONT_DATE'].head()

0    2453403.5
1    2453137.5
2    2453156.5
3    2453189.5
4    2453189.5
Name: CONT_DATE, dtype: float64

In [124]:
#To make these dates and times easier to manage, let's convert them to datetime. We can add new columns DISCOVERY_DATETIME and CONTAINMENT_DATETIME.
df['DISCOVERY_DATETIME'] = df['DISCOVERY_DATE'];
df['CONT_DATETIME'] = df['CONT_DATE'];

In [125]:
df['DISCOVERY_DATETIME'].head()

0    2453403.5
1    2453137.5
2    2453156.5
3    2453184.5
4    2453184.5
Name: DISCOVERY_DATETIME, dtype: float64

In [126]:
#To populate those two rows, let's convert them into datetime.
df['DISCOVERY_DATETIME'] = df['DISCOVERY_DATETIME'].apply(lambda x: julian.from_jd(x, fmt="jd"))
df['CONT_DATETIME'] = df['CONT_DATETIME'].apply(lambda x:julian.from_jd(x, fmt="jd"))


In [127]:
df['DISCOVERY_DATETIME'].head()

0   2005-02-02
1   2004-05-12
2   2004-05-31
3   2004-06-28
4   2004-06-28
Name: DISCOVERY_DATETIME, dtype: datetime64[ns]

In [128]:
df['CONT_DATETIME'].head()

0   2005-02-02
1   2004-05-12
2   2004-05-31
3   2004-07-03
4   2004-07-03
Name: CONT_DATETIME, dtype: datetime64[ns]

In [129]:
#Let's also add the time 
temp_df = pd.DataFrame();
temp_df['dt'] = df['DISCOVERY_TIME'].apply(lambda x: dt.timedelta(hours=int(x[0:2]), minutes=int(x[2:5])))
df['DISCOVERY_DATETIME'] = df['DISCOVERY_DATETIME'] + temp_df['dt']
df['DISCOVERY_DATETIME'].head()

0   2005-02-02 13:00:00
1   2004-05-12 08:45:00
2   2004-05-31 19:21:00
3   2004-06-28 16:00:00
4   2004-06-28 16:00:00
Name: DISCOVERY_DATETIME, dtype: datetime64[ns]

In [131]:
#Do the same thing for CONT_DATETIME
temp_df = pd.DataFrame();
temp_df['dt'] = df['CONT_TIME'].apply(lambda x: dt.timedelta(hours=int(x[0:2]), minutes=int(x[2:5])))
df['CONT_DATETIME'] = df['CONT_DATETIME'] + temp_df['dt']
df['CONT_DATETIME'].head()

0   2005-02-03 11:00:00
1   2004-05-13 07:00:00
2   2004-06-01 16:48:00
3   2004-07-04 04:00:00
4   2004-07-04 00:00:00
Name: CONT_DATETIME, dtype: datetime64[ns]

In [133]:
df.to_csv("data_with_target.csv")
#Checkpoint to save the data with the above columns before reloading

In [139]:
df = pd.read_csv("data_with_target.csv")

/Applications/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (9,11,12,13,14,15,16,17,18,19,36,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [140]:
df['CONT_DATETIME'] = df['CONT_DATETIME'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
judf['DISCOVERY_DATETIME'] = df['DISCOVERY_DATETIME'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df['TIME_TO_CONT'] = 0;
df['TIME_TO_CONT'] = df['CONT_DATETIME'] - df['DISCOVERY_DATETIME']

In [141]:
df['TIME_TO_CONT'].describe()


count                     892007
mean      1 days 22:18:16.178348
std      13 days 19:29:31.562665
min              0 days 00:01:00
25%              0 days 15:24:00
50%              0 days 18:40:00
75%              0 days 23:59:00
max           3653 days 18:45:00
Name: TIME_TO_CONT, dtype: object

In [143]:
df.to_csv("data_with_target.csv")

In [146]:
df['TIME_TO_CONT'].head()

0   0 days 22:00:00
1   0 days 22:15:00
2   0 days 21:27:00
3   5 days 12:00:00
4   5 days 08:00:00
Name: TIME_TO_CONT, dtype: timedelta64[ns]

In [144]:
df.to_pickle("data_with_target.pkl")